In [1]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("churn-7043.csv")

In [2]:
df=df.drop(['customerID'], axis=1) 
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)




#BOX-COX
#lemda=0.5
#num_df=(num_df**lemda)
#num_df=num_df-1
#num_df[num_df < 0]=0
#num_df=num_df.div(lemda)
#End BOX-COX
#Z-score
num_df=(num_df-num_df.min())/(num_df.std(ddof=0)) ##Z-score

#Log
#num_df=round(np.log(num_df.add(1)),2)
#num_df=num_df.replace([np.inf, -np.inf], np.nan)
#End Log


result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)


result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df

X=result_df_op
y=churn

from sklearn.model_selection import train_test_split  
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train=X
X_test=X
y_train=y
y_test=y

In [3]:
X.shape

(7043, 19)

In [4]:
X_train=X
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=15).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
18,TotalCharges,1616.483021
15,Contract,1115.780167
1,tenure,662.883693
9,OnlineSecurity,551.611529
12,TechSupport,523.303866
0,SeniorCitizen,364.505842
10,OnlineBackup,230.086520
11,DeviceProtection,191.303140
2,MonthlyCharges,170.335348
5,Dependents,133.036443


In [5]:
y_train=y
x_train_chi = select_feature.transform(X)
x_train_chi.shape

(7043, 15)

In [6]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

3950 1224 495 1374
pod:  0.7351524879614767
pof:  0.23656745264785467
AUC:  0.749292517656811


In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 15 folds for each of 100 candidates, totalling 1500 fits
{'var_smoothing': 5.336699231206313e-07}
GaussianNB(var_smoothing=5.336699231206313e-07)
0.7894827892122073


In [8]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=3.5111917342151277e-07)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

4348 826 719 1150
pod:  0.6153023006955591
pof:  0.15964437572477774
AUC:  0.7278289624853906
accuracy:  0.7806332528751952


In [9]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

accuracy:  0.54812


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits
{'C': 3, 'multi_class': 'ovr', 'penalty': 'l1', 'solver': 'liblinear'}
LogisticRegression(C=3, multi_class='ovr', penalty='l1', solver='liblinear')
0.8006515957446808


In [11]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', verbose=0,
                   warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4646 528 884 985
pod:  0.527019796682718
pof:  0.10204870506378044
AUC:  0.7124855458094689
accuracy:  0.7995172511713758


In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 10 folds for each of 16 candidates, totalling 160 fits
{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}
KNeighborsClassifier(metric='manhattan', n_neighbors=19, weights='distance')
0.7555018939393939


In [13]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(metric='manhattan', weights='distance', n_neighbors=19 )  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


4918 256 1466 403
pod:  0.21562332798287853
pof:  0.04947816003092385
AUC:  0.5830725839759774
accuracy:  0.7555019167968196


In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 288 candidates, totalling 2880 fits
{'bootstrap': True, 'max_depth': 80, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 12, 'n_estimators': 200}
RandomForestClassifier(max_depth=80, max_features=3, min_samples_leaf=3,
                       min_samples_split=12, n_estimators=200)
0.8043427627337202


In [15]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(max_depth=80, max_features=3, min_samples_leaf=5,
                       min_samples_split=8, n_estimators=300)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4721 453 947 922
pod:  0.4933119315141787
pof:  0.08755315036722072
AUC:  0.702879390573479
accuracy:  0.8012210705665199


In [16]:
# Decision Tree
import time
start = time.time()
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 


#param_grid={
#    'criterion': ['gini', 'entropy'],
#    'splitter': ['best', 'random'],
#    'min_samples_split' : range(10,500,20),
#    'max_depth': [1, 2, 3, 4, 5, 20],
#    'max_features': [1, 2, 3, 4],
#    'max_leaf_nodes': list(range(2, 100)), 
#    'min_samples_split': [2, 3, 4]
#} 

param_grid={
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
} 

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)
end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
{'criterion': 'gini', 'splitter': 'random'}
DecisionTreeClassifier(splitter='random')
0.7333540860735009
Time:  0.21465778350830078


In [17]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
classifier =DecisionTreeClassifier(criterion='entropy', splitter='random')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4167 1007 942 927
pod:  0.4959871589085072
pof:  0.19462698105914186
AUC:  0.6506800889246827
accuracy:  0.7232713332386767


In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

grid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=10, n_jobs=-1)

grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

{'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'n_estimators': 10, 'subsample': 0.85}
GradientBoostingClassifier(learning_rate=0.2, max_depth=8, max_features='sqrt',
                           min_samples_leaf=0.1, min_samples_split=0.1,
                           n_estimators=10, subsample=0.85)
0.7944070357833656


In [20]:
#  Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
classifier =GradientBoostingClassifier(learning_rate=0.2, max_depth=8, max_features='sqrt',
                           min_samples_leaf=0.1, min_samples_split=0.1,
                           n_estimators=10, subsample=0.85)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4903 271 1218 651
pod:  0.34831460674157305
pof:  0.05237727097023579
AUC:  0.6479686678856686
accuracy:  0.7885844100525344


In [21]:
import time
start = time.time()

# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
 
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense
# Function to create model, required for KerasClassifier
def create_model(): 
    #model = Sequential()
    #model.add(Dense(12, input_shape=(8,), activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model = Sequential()
    #get number of columns in training data
    n_cols = x_train_chi.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
def nn_cl_fun():
        #opt = Adam(lr = learning_rate)
        n_cols = x_train_chi.shape[1]
        nn = Sequential()
        nn.add(Dense(10, input_dim=n_cols, activation='relu'))
        nn.add(Dense(250, activation='relu'))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return nn
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
#model = KerasClassifier(model=create_model, verbose=0)
model = KerasClassifier(build_fn=nn_cl_fun, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train_chi, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

end = time.time()
time_taken = end - start
print('Time: ',time_taken)  

Best: 0.784468 using {'batch_size': 20, 'epochs': 100}
0.732649 (0.071452) with: {'batch_size': 10, 'epochs': 10}
0.744287 (0.033041) with: {'batch_size': 10, 'epochs': 50}
0.782339 (0.014783) with: {'batch_size': 10, 'epochs': 100}
0.767851 (0.018658) with: {'batch_size': 20, 'epochs': 10}
0.678956 (0.090946) with: {'batch_size': 20, 'epochs': 50}
0.784468 (0.008196) with: {'batch_size': 20, 'epochs': 100}
0.766431 (0.020779) with: {'batch_size': 40, 'epochs': 10}
0.737179 (0.037063) with: {'batch_size': 40, 'epochs': 50}
0.758631 (0.034387) with: {'batch_size': 40, 'epochs': 100}
0.614669 (0.183641) with: {'batch_size': 60, 'epochs': 10}
0.727382 (0.037003) with: {'batch_size': 60, 'epochs': 50}
0.771123 (0.015005) with: {'batch_size': 60, 'epochs': 100}
0.736617 (0.006276) with: {'batch_size': 80, 'epochs': 10}
0.741586 (0.007382) with: {'batch_size': 80, 'epochs': 50}
0.756067 (0.021669) with: {'batch_size': 80, 'epochs': 100}
0.583579 (0.211211) with: {'batch_size': 100, 'epochs':

In [24]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

import time
start = time.time()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2,  epochs=100, batch_size= 20, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix


from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 

end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Epoch 1/100
254/254 [==============================] - 2s 5ms/step - loss: 7.3580 - accuracy: 0.6142 - val_loss: 0.6814 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 0.8586 - accuracy: 0.6733 - val_loss: 0.5971 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6243 - accuracy: 0.7241 - val_loss: 0.5937 - val_accuracy: 0.7279
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6110 - accuracy: 0.7283 - val_loss: 0.6060 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5920 - accuracy: 0.7342 - val_loss: 0.6128 - val_accuracy: 0.7279
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5925 - accuracy: 0.7295 - val_loss: 0.5836 - val_accuracy: 0.7279
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5975 - accuracy: 0.7209 - val_loss: 0.5846 - val_accuracy: 0.7287

254/254 [==============================] - 2s 4ms/step - loss: 7.4706 - accuracy: 0.6117 - val_loss: 0.9353 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 1.0437 - accuracy: 0.6526 - val_loss: 0.5837 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.7276 - accuracy: 0.6914 - val_loss: 0.5784 - val_accuracy: 0.7279
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6394 - accuracy: 0.7094 - val_loss: 0.5753 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5427 - accuracy: 0.7234 - val_loss: 0.4789 - val_accuracy: 0.7492
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5469 - accuracy: 0.7240 - val_loss: 0.5573 - val_accuracy: 0.7279
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5216 - accuracy: 0.7267 - val_loss: 0.5420 - val_accuracy: 0.7279
Epoch 8/100

254/254 [==============================] - 2s 4ms/step - loss: 10.7411 - accuracy: 0.6202 - val_loss: 1.6369 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 1.1820 - accuracy: 0.6479 - val_loss: 0.8441 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.7184 - accuracy: 0.6910 - val_loss: 0.5770 - val_accuracy: 0.7279
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6078 - accuracy: 0.7187 - val_loss: 0.5876 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6070 - accuracy: 0.7284 - val_loss: 0.5878 - val_accuracy: 0.7279
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5768 - accuracy: 0.7344 - val_loss: 0.5842 - val_accuracy: 0.7279
y2_pred:  [[2.4839911e-01]
 [2.6054120e-01]
 [2.9568011e-01]
 [2.2782737e-01]
 [2.5469220e-01]
 [2.2420910e-01]
 [2.5735587e-01]
 [2.1225679e-01]
 [2.262

254/254 [==============================] - 2s 5ms/step - loss: 18.0161 - accuracy: 0.6003 - val_loss: 1.1169 - val_accuracy: 0.3320
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 1.6677 - accuracy: 0.6362 - val_loss: 0.6178 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 4ms/step - loss: 0.8244 - accuracy: 0.6842 - val_loss: 0.5777 - val_accuracy: 0.7279
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6193 - accuracy: 0.7095 - val_loss: 0.6311 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5937 - accuracy: 0.7374 - val_loss: 0.7518 - val_accuracy: 0.7279
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6203 - accuracy: 0.7206 - val_loss: 0.6728 - val_accuracy: 0.5481
y2_pred:  [[4.2926025e-01]
 [5.3052396e-01]
 [4.9048141e-01]
 [3.9514109e-01]
 [5.9333402e-01]
 [3.2479674e-01]
 [4.0446353e-01]
 [6.0467911e-01]
 [4.267

254/254 [==============================] - 2s 5ms/step - loss: 13.4653 - accuracy: 0.5993 - val_loss: 1.2366 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 4ms/step - loss: 0.9378 - accuracy: 0.6493 - val_loss: 0.6196 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 4ms/step - loss: 0.8165 - accuracy: 0.6902 - val_loss: 0.6992 - val_accuracy: 0.4763
Epoch 4/100
254/254 [==============================] - 1s 4ms/step - loss: 0.6437 - accuracy: 0.7054 - val_loss: 0.6603 - val_accuracy: 0.7287
Epoch 5/100
254/254 [==============================] - 1s 4ms/step - loss: 0.6108 - accuracy: 0.7334 - val_loss: 0.5829 - val_accuracy: 0.7279
Epoch 6/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5835 - accuracy: 0.7368 - val_loss: 0.5819 - val_accuracy: 0.7279
Epoch 7/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5804 - accuracy: 0.7392 - val_loss: 0.5824 - val_accuracy: 0.7279
Epoch 8/10

254/254 [==============================] - 2s 5ms/step - loss: 17.5770 - accuracy: 0.5917 - val_loss: 1.4070 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 4ms/step - loss: 1.3493 - accuracy: 0.6412 - val_loss: 0.5966 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 4ms/step - loss: 0.6274 - accuracy: 0.7312 - val_loss: 0.5918 - val_accuracy: 0.7279
Epoch 4/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5835 - accuracy: 0.7421 - val_loss: 0.5886 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5881 - accuracy: 0.7349 - val_loss: 0.5819 - val_accuracy: 0.7279
Epoch 6/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5762 - accuracy: 0.7396 - val_loss: 0.5860 - val_accuracy: 0.7287
Epoch 7/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5786 - accuracy: 0.7408 - val_loss: 0.5910 - val_accuracy: 0.7287
Epoch 8/10

254/254 [==============================] - 2s 5ms/step - loss: 11.7894 - accuracy: 0.5967 - val_loss: 1.6492 - val_accuracy: 0.2847
Epoch 2/100
254/254 [==============================] - 1s 4ms/step - loss: 1.1388 - accuracy: 0.6307 - val_loss: 0.6991 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 4ms/step - loss: 0.7399 - accuracy: 0.6812 - val_loss: 0.6017 - val_accuracy: 0.7279
Epoch 4/100
254/254 [==============================] - 1s 4ms/step - loss: 0.6422 - accuracy: 0.7116 - val_loss: 0.6010 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 4ms/step - loss: 0.6033 - accuracy: 0.7316 - val_loss: 0.6062 - val_accuracy: 0.7279
Epoch 6/100
254/254 [==============================] - 1s 4ms/step - loss: 0.6212 - accuracy: 0.7168 - val_loss: 0.5854 - val_accuracy: 0.7279
Epoch 7/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5791 - accuracy: 0.7386 - val_loss: 0.5814 - val_accuracy: 0.7279
Epoch 8/10

254/254 [==============================] - 2s 5ms/step - loss: 15.4444 - accuracy: 0.6195 - val_loss: 1.1252 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 4ms/step - loss: 1.4549 - accuracy: 0.6281 - val_loss: 3.1673 - val_accuracy: 0.2957
Epoch 3/100
254/254 [==============================] - 1s 4ms/step - loss: 0.9706 - accuracy: 0.6728 - val_loss: 1.0131 - val_accuracy: 0.3549
Epoch 4/100
254/254 [==============================] - 1s 4ms/step - loss: 0.8952 - accuracy: 0.6575 - val_loss: 0.5833 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 4ms/step - loss: 0.6082 - accuracy: 0.7127 - val_loss: 0.6020 - val_accuracy: 0.7279
Epoch 6/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5835 - accuracy: 0.7352 - val_loss: 0.5574 - val_accuracy: 0.7287
Epoch 7/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5886 - accuracy: 0.7225 - val_loss: 0.5385 - val_accuracy: 0.7279
Epoch 8/10

254/254 [==============================] - 2s 5ms/step - loss: 12.0057 - accuracy: 0.6151 - val_loss: 1.9592 - val_accuracy: 0.7248
Epoch 2/100
254/254 [==============================] - 1s 4ms/step - loss: 0.8019 - accuracy: 0.6620 - val_loss: 0.7522 - val_accuracy: 0.4093
Epoch 3/100
254/254 [==============================] - 1s 4ms/step - loss: 0.7507 - accuracy: 0.6958 - val_loss: 1.0317 - val_accuracy: 0.3675
Epoch 4/100
254/254 [==============================] - 1s 4ms/step - loss: 0.6947 - accuracy: 0.6807 - val_loss: 0.8024 - val_accuracy: 0.7248
Epoch 5/100
254/254 [==============================] - 1s 4ms/step - loss: 0.6710 - accuracy: 0.7061 - val_loss: 0.6890 - val_accuracy: 0.7248
Epoch 6/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5952 - accuracy: 0.7375 - val_loss: 0.5772 - val_accuracy: 0.7256
Epoch 7/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5880 - accuracy: 0.7347 - val_loss: 0.5975 - val_accuracy: 0.7256
Epoch 8/10

254/254 [==============================] - 2s 5ms/step - loss: 9.8584 - accuracy: 0.6115 - val_loss: 0.6127 - val_accuracy: 0.7248
Epoch 2/100
254/254 [==============================] - 1s 4ms/step - loss: 0.8542 - accuracy: 0.6409 - val_loss: 0.6912 - val_accuracy: 0.7248
Epoch 3/100
254/254 [==============================] - 1s 4ms/step - loss: 0.6149 - accuracy: 0.7388 - val_loss: 0.7276 - val_accuracy: 0.4487
Epoch 4/100
254/254 [==============================] - 1s 4ms/step - loss: 0.7003 - accuracy: 0.7035 - val_loss: 0.5993 - val_accuracy: 0.7248
Epoch 5/100
254/254 [==============================] - 1s 4ms/step - loss: 0.6407 - accuracy: 0.7108 - val_loss: 0.5883 - val_accuracy: 0.7240
Epoch 6/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5755 - accuracy: 0.7400 - val_loss: 0.6047 - val_accuracy: 0.7248
Epoch 7/100
254/254 [==============================] - 1s 4ms/step - loss: 0.5829 - accuracy: 0.7386 - val_loss: 0.5869 - val_accuracy: 0.7240
Epoch 8/100

In [ ]:
##RNN Hyperparemeter tuning
import time
start = time.time()

# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
 
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM, GRU
from keras.layers import Embedding
# Function to create model, required for KerasClassifier
def create_model(): 
    #model = Sequential()
    #model.add(Dense(12, input_shape=(8,), activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model = Sequential()
    #get number of columns in training data
    #n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
def nn_cl_fun():
        #opt = Adam(lr = learning_rate)
        #n_cols = X2_train.shape[1]
        #nn = Sequential()
        #nn.add(Dense(10, input_dim=n_cols, activation='relu'))
        #nn.add(Dense(250, activation='relu'))
        #nn.add(Dense(1, activation='sigmoid'))
        #nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model = Sequential()
        model.add(Embedding(10000, 32))
        model.add(LSTM(32))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
        return model
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
#model = KerasClassifier(model=create_model, verbose=0)
model = KerasClassifier(build_fn=nn_cl_fun, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train_chi, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

end = time.time()
time_taken = end - start
print('Time: ',time_taken)  

In [23]:
#RNN
import time
start = time.time()

def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=10, batch_size=100, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)

end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Epoch 1/10
51/51 [==============================] - 7s 29ms/step - loss: 0.6180 - acc: 0.6986 - val_loss: 0.5462 - val_acc: 0.7279
Epoch 2/10
51/51 [==============================] - 1s 16ms/step - loss: 0.5316 - acc: 0.7309 - val_loss: 0.5253 - val_acc: 0.7295
Epoch 3/10
51/51 [==============================] - 1s 16ms/step - loss: 0.5019 - acc: 0.7413 - val_loss: 0.5162 - val_acc: 0.7319
Epoch 4/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4990 - acc: 0.7392 - val_loss: 0.5141 - val_acc: 0.7311
Epoch 5/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4833 - acc: 0.7430 - val_loss: 0.5121 - val_acc: 0.7366
Epoch 6/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4692 - acc: 0.7656 - val_loss: 0.5072 - val_acc: 0.7342
Epoch 7/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4459 - acc: 0.7796 - val_loss: 0.5068 - val_acc: 0.7334
Epoch 8/10
51/51 [==============================] - 1s 19ms/step - loss: 0.3963 - a

Epoch 1/10
51/51 [==============================] - 3s 29ms/step - loss: 0.6117 - acc: 0.7334 - val_loss: 0.5510 - val_acc: 0.7279
Epoch 2/10
51/51 [==============================] - 1s 16ms/step - loss: 0.5280 - acc: 0.7327 - val_loss: 0.5220 - val_acc: 0.7303
Epoch 3/10
51/51 [==============================] - 1s 16ms/step - loss: 0.5073 - acc: 0.7381 - val_loss: 0.5145 - val_acc: 0.7295
Epoch 4/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4943 - acc: 0.7377 - val_loss: 0.5115 - val_acc: 0.7311
Epoch 5/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4821 - acc: 0.7456 - val_loss: 0.5246 - val_acc: 0.7350
Epoch 6/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4598 - acc: 0.7734 - val_loss: 0.5092 - val_acc: 0.7366
Epoch 7/10
51/51 [==============================] - 1s 19ms/step - loss: 0.4316 - acc: 0.7990 - val_loss: 0.4999 - val_acc: 0.7461
Epoch 8/10
51/51 [==============================] - 1s 18ms/step - loss: 0.3524 - a

Epoch 1/10
51/51 [==============================] - 3s 27ms/step - loss: 0.6049 - acc: 0.7293 - val_loss: 0.5375 - val_acc: 0.7287
Epoch 2/10
51/51 [==============================] - 1s 16ms/step - loss: 0.5304 - acc: 0.7322 - val_loss: 0.5217 - val_acc: 0.7295
Epoch 3/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4985 - acc: 0.7455 - val_loss: 0.5147 - val_acc: 0.7334
Epoch 4/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4937 - acc: 0.7412 - val_loss: 0.5109 - val_acc: 0.7326
Epoch 5/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4913 - acc: 0.7412 - val_loss: 0.5109 - val_acc: 0.7366
Epoch 6/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4637 - acc: 0.7676 - val_loss: 0.5189 - val_acc: 0.7421
Epoch 7/10
51/51 [==============================] - 1s 19ms/step - loss: 0.4404 - acc: 0.7890 - val_loss: 0.5148 - val_acc: 0.7342
Epoch 8/10
51/51 [==============================] - 1s 18ms/step - loss: 0.3872 - a

Epoch 1/10
51/51 [==============================] - 3s 27ms/step - loss: 0.6046 - acc: 0.6968 - val_loss: 0.5368 - val_acc: 0.7311
Epoch 2/10
51/51 [==============================] - 1s 16ms/step - loss: 0.5298 - acc: 0.7285 - val_loss: 0.5225 - val_acc: 0.7319
Epoch 3/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4886 - acc: 0.7519 - val_loss: 0.5155 - val_acc: 0.7287
Epoch 4/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4824 - acc: 0.7523 - val_loss: 0.5104 - val_acc: 0.7326
Epoch 5/10
51/51 [==============================] - 1s 15ms/step - loss: 0.4844 - acc: 0.7443 - val_loss: 0.5084 - val_acc: 0.7326
Epoch 6/10
51/51 [==============================] - 1s 14ms/step - loss: 0.4532 - acc: 0.7763 - val_loss: 0.5071 - val_acc: 0.7366
Epoch 7/10
51/51 [==============================] - 1s 13ms/step - loss: 0.4208 - acc: 0.8023 - val_loss: 0.5200 - val_acc: 0.7326
Epoch 8/10
51/51 [==============================] - 1s 14ms/step - loss: 0.3643 - a

Epoch 1/10
51/51 [==============================] - 4s 32ms/step - loss: 0.6031 - acc: 0.7287 - val_loss: 0.5357 - val_acc: 0.7248
Epoch 2/10
51/51 [==============================] - 1s 17ms/step - loss: 0.5194 - acc: 0.7338 - val_loss: 0.5218 - val_acc: 0.7216
Epoch 3/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4827 - acc: 0.7506 - val_loss: 0.5193 - val_acc: 0.7192
Epoch 4/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4837 - acc: 0.7476 - val_loss: 0.5232 - val_acc: 0.7192
Epoch 5/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4828 - acc: 0.7491 - val_loss: 0.5144 - val_acc: 0.7208
Epoch 6/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4446 - acc: 0.7772 - val_loss: 0.5169 - val_acc: 0.7326
Epoch 7/10
51/51 [==============================] - 1s 18ms/step - loss: 0.4179 - acc: 0.8188 - val_loss: 0.5060 - val_acc: 0.7366
Epoch 8/10
51/51 [==============================] - 1s 17ms/step - loss: 0.3458 - a